In [ ]:
"""
processed_ds = DataProcessor()

subset_trainer = SubsetTrainer(
                                training_args=training_args,
                                valid_ds=processed_ds["valid"]
                                test_ds=processed_ds["test"]
                                )

subset_searcher = SubsetSearcher(
                            subset_trainer=subset_trainer,
                            ...subset args
)

subset_searcher.search()
"""